In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys
import os
from tqdm.notebook import tqdm

from PIL import Image

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

System Version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
PyTorch version 2.4.0+cpu
Torchvision version 0.19.0+cpu
Numpy version 1.26.4
Pandas version 2.2.2


# CSV Cleansing

In [2]:
csv_path='/kaggle/input/fundusimage/ocular-disease-recognition-odir5k/full_df.csv'

In [3]:
csv = pd.read_csv(csv_path)
csv.shape


(6392, 19)

In [4]:
# csv.head()

# Dataset

In [5]:
train_path='/kaggle/input/fundusimage/ocular-disease-recognition-odir5k/ODIR-5K/Training Images'
test_path='/kaggle/input/fundusimage/ocular-disease-recognition-odir5k/ODIR-5K/Testing Images'

In [6]:
IMAGE_SIZE = 128
data_transform = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), 
                                             transforms.ToTensor()])

In [7]:
class OcularDiseaseDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file) 
        self.transform = transform 
        
        # Define a mapping from string labels to integers
        self.label_mapping = {
            "N": 0,  # Normal
            "D": 1,  # Diabetes
            "G": 2,  # Glaucoma
            "C": 3,  # Cataract
            "A": 4,  # Age-related Macular Degeneration
            "H": 5,  # Hypertension
            "M": 6,  # Pathological Myopia
            "O": 7   # Other diseases/abnormalities
        }

    def __len__(self):
        return len(self.df)  

    def __getitem__(self, index):
        image_path = self.df.filepath[index].replace('../input', '/kaggle/input/fundusimage')
        image = Image.open(image_path)
        
        # Process the label, which is stored as a string like '["N"]'
        label = self.df.labels[index]
        label = label.strip("[]").strip("'\"")  # Clean the string
        
        # Convert the label to the corresponding integer using the label_mapping
        label = self.label_mapping.get(label, -1)  # Use -1 for any unknown labels
        
        # Convert label to a tensor
        label = torch.tensor(label, dtype=torch.long)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label



In [8]:
train_dataset = OcularDiseaseDataset(
    csv_path,
    data_transform
)

In [9]:
len(train_dataset)

6392

In [10]:
image, label = train_dataset[3274]
print(label)


tensor(2)


    "N": 0,  # Normal
    "D": 1,  # Diabetes
    "G": 2,  # Glaucoma
    "C": 3,  # Cataract
    "A": 4,  # Age-related Macular Degeneration
    "H": 5,  # Hypertension
    "M": 6,  # Pathological Myopia
    "O": 7   # Other diseases/abnormalities

In [11]:
target_to_class = ['Normal', 'Diabetes', 'Glaucoma', 'Cataract', 'Age', 'Hypertension',
                  'Pathological Myopia', 'Other']

target_to_class[label]

'Glaucoma'

# Dataloader

In [12]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [13]:
for images, labels in train_dataloader:
    break
    
images.shape, labels.shape

(torch.Size([32, 3, 128, 128]), torch.Size([32]))